In [1]:
import numpy as np
import scipy.stats as sts
import warnings
#warnings.filterwarnings('ignore')

In [2]:
#readin datasets
#columns: c, k, w, r
df = np.loadtxt('data/MacroSeries.txt', delimiter=",")


In [3]:
'''
--------------------------------------------------------------------------
Question (a)
--------------------------------------------------------------------------
Steps:
1. Design a function to return the pdf of a normal distribution
2. Design a maximum likelihood
3. Design a critierion function


'''

def ar_norm_dist(z_vec, rho, mu, sigma):
    '''
    --------------------------------------------------------------------------
    Function to return the pdf of a normal distribution based on AR(1) process
    --------------------------------------------------------------------------
    Input: 
    z_vec : TPF shocks
    rho:    AR coefficient
    mu:     mean of TPF shocks
    sigma:  std of TPF shocks

    Target output:
    pdf_vals




    '''
    mean_vec = rho*z_vec[:-1] + (1 - rho)*mu 
    mean_vec = np.append(mu, mean_vec)                # create the vector of means of normal distribution
    pdf_vals    = ((1/(sigma * np.sqrt(2 * np.pi)) *
                    np.exp( - (z_vec - mean_vec)**2 / (2 * sigma**2))))
    
    return pdf_vals


In [4]:
def log_likelihood(w_vec, k_vec, alpha, rho, mu, sigma):
    '''
    -------------------------------------------------------------------------
    Function to return the loglikelihood of question based on w and k 
    -------------------------------------------------------------------------
    Input:
    w_vec:          wage per unit of labor
    k_vec:          saving
    alpha:          
    rho:
    mu:
    sigma: 
    
    '''
    # we need to back out the z series
    z_vec = np.log(w_vec) - alpha*np.log(k_vec) - np.log(1 - alpha)
    pdf_vals = ar_norm_dist(z_vec, rho, mu, sigma)
    ln_pdf_vals = np.log(pdf_vals)
    log_lik_val = ln_pdf_vals.sum()
    return log_lik_val

def log_likelihood2(r_vec, k_vec, alpha, rho, mu, sigma):
    '''
    -------------------------------------------------------------------------
    Function to return the loglikelihood of question based on r and k 
    -------------------------------------------------------------------------
    Input:
    r_vec:          interest rate
    k_vec:          saving
    alpha:          
    rho:
    mu:
    sigma: 
    
    '''
    # we need to back out the z series
    z_vec = np.log(r_vec) +(1 - alpha)*np.log(k_vec) - np.log(alpha)
    pdf_vals = ar_norm_dist(z_vec, rho, mu, sigma)
    ln_pdf_vals = np.log(pdf_vals)
    log_lik_val = ln_pdf_vals.sum()
    return log_lik_val

def crit(params, *args):
    '''
    -------------------------------------------------------------------------
    Function to derive MLE estimation of parameters of loglikelihood function
    -------------------------------------------------------------------------
    Inputs:
    
    Outputs:
    
    '''
    alpha, rho, mu, sigma = params
    w_r_vec, k_vec, indicator = args
    if indicator == 1:
        log_lik_val = log_likelihood(w_r_vec, k_vec, alpha, rho, mu, sigma)
    elif indicator ==2:
        log_lik_val = log_likelihood2(w_r_vec, k_vec, alpha, rho, mu, sigma)
    
    neg_log_lik_val = -log_lik_val
    
    return neg_log_lik_val

In [5]:
# MLE estimation
import scipy.optimize as opt

# Instead of checking one set of initial parameters, we will test a whole sets of them and returned the one with the biggest value
alpha_vec = np.linspace(1e-5, (1 - 1e-5), 10)
rho_vec = np.linspace((1e-5 - 1), (1 - 1e-5), 10)
mu_vec = np.linspace(1e-5, 20, 10)
sigma_vec = np.linspace(1e-5, 20, 10)
mle_args = (df[:, 2], df[:, 1], 1)
mle2_args = (df[:, 3], df[:, 1], 2)
bnds = ((1e-5, 1-(1e-5)), ((1e-5)-1, 1-(1e-5)), (1e-5, None), (1e-5, None))
mle_fun = 1e10
mle_params = None
mle_results = None

def init_finder(alpha_vec, rho_vec, mu_vec, sigma_vec, mle_args, bnds, mle_fun, mle_params, mle_results):
    mle_fun = mle_fun
    mle_params = mle_params
    mle_results = mle_results
    for alpha_init in alpha_vec:
        for rho_init in rho_vec:
            for mu_init in mu_vec:
                for sigma_init in sigma_vec:
                    params_init = (alpha_init, rho_init, mu_init, sigma_init)
                    results = opt.minimize(crit, params_init, args=(mle_args), method='L-BFGS-B',
                           bounds= bnds)
                    if results.fun < mle_fun:
                        mle_results = results
                        mle_fun = results.fun
                        mle_params = results.x
                        alpha0  = alpha_init
                        rho0 = rho_init
                        mu0 = mu_init
                        sigma0 = sigma_init
    return mle_results, alpha0, rho0, mu0, sigma0

final_results, alpha0, rho0, mu0, sigma0 = init_finder(alpha_vec, rho_vec, mu_vec, sigma_vec, mle_args, bnds, mle_fun, mle_params, mle_results)
print("Results for question (a):")
print(final_results)
print(alpha0, rho0, mu0, sigma0)
final_results2, alpha0_2, rho0_2, mu0_2, sigma0_2 = init_finder(alpha_vec, rho_vec, mu_vec, sigma_vec, mle2_args, bnds, mle_fun, mle_params, mle_results)
print("Results for question (b):")
print(final_results)
print(alpha0_2, rho0_2, mu0_2, sigma0_2)

C:\Users\Lenovo\Anaconda3\lib\site-packages\ipykernel\__main__.py:18: RuntimeWarning: divide by zero encountered in log
C:\Users\Lenovo\Anaconda3\lib\site-packages\scipy\optimize\optimize.py:628: RuntimeWarning: invalid value encountered in double_scalars
  grad[k] = (f(*((xk + d,) + args)) - f0) / d[k]


Results for question (a):
      fun: -96.706908090043953
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 0.00726885,  0.00053859,  0.00042917, -0.00304681])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 300
      nit: 34
   status: 0
  success: True
        x: array([ 0.45756255,  0.72045842,  9.52207362,  0.0919961 ])
0.888881111111 0.55555 1e-05 11.1111155556


C:\Users\Lenovo\Anaconda3\lib\site-packages\ipykernel\__main__.py:39: RuntimeWarning: divide by zero encountered in log


Results for question (b):
      fun: -96.706908090043953
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 0.00726885,  0.00053859,  0.00042917, -0.00304681])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 300
      nit: 34
   status: 0
  success: True
        x: array([ 0.45756255,  0.72045842,  9.52207362,  0.0919961 ])
0.222227777778 0.77777 20.0 11.1111155556


In [7]:
#problem 3
# we firstly calculated the z* when r = 1 based on equation (4)

r = 1
k = 7500000
alpha, rho, mu, sigma = final_results.x
z_star = np.log(r) + (1 - alpha)*np.log(k) - np.log(alpha)
mean = rho*10 + (1 - rho)*mu
prob = 1 - sts.norm.cdf(z_star, mean, sigma)
print("The probability that the interest rate this period will be greater than 1 is : ", prob)


The probability that the interest rate this period will be greater than 1 is :  0.999999968199


In [50]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y
